In [1]:
import os
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sps
import implicit
import pickle
from implicit.evaluation import train_test_split, precision_at_k, mean_average_precision_at_k

In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
    names=['user_id', 'movie_id', 'rating', 'timestamp'], 
    usecols=['user_id', 'movie_id', 'rating','timestamp'],
    engine='python')
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
# user_id_list = ratings['user_id'].values
# item_id_list = ratings['movie_id'].values
# rating_list = ratings['rating'].values
# timestamp_list = ratings['timestamp'].values

In [4]:
ratings['rating']=1
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,1,978300760
1,1,661,1,978302109
2,1,914,1,978301968
3,1,3408,1,978300275
4,1,2355,1,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,1,956704887
1000206,6040,562,1,956704746
1000207,6040,1096,1,956715648


In [5]:
df_user_movie_rating= ratings.pivot(
    index='user_id',
    columns='movie_id',
    values='rating'
).fillna(0)
df_user_movie_rating

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_user_movie_timestamp= ratings.pivot(
    index='user_id',
    columns='movie_id',
    values='timestamp'
).fillna(0)
df_user_movie_timestamp

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,978824268.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,978245916.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,956753920.0,0.0,956717530.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
import numpy as np

df_user_movie_rating_mtx = df_user_movie_rating.values
df_user_movie_timestamp_mtx = df_user_movie_timestamp.values

In [8]:
# df_user_movie_rating
df_user_movie_rating_mtx
print(df_user_movie_rating_mtx.shape[0])

6040


In [9]:
from IncrementalSparseMatrix import IncrementalSparseMatrix

In [10]:
import os
os.getcwd() 

'/Users/macpro/Recsys/Anaelle/Data_manager'

In [11]:
def split_data_on_timestamp(URM, URM_timestamp):

    URM_all=sps.csr_matrix(URM)
    print(URM_all.shape)
    URM_timestamp=sps.csr_matrix(URM_timestamp)

    n_rows, n_cols = URM_all.shape


    URM_train_builder = IncrementalSparseMatrix(n_rows=n_rows, n_cols=n_cols)
    URM_test_builder = IncrementalSparseMatrix(n_rows=n_rows, n_cols=n_cols)
    URM_validation_builder = IncrementalSparseMatrix(n_rows=n_rows, n_cols=n_cols)
    

    all_items = np.arange(0, n_cols, dtype=np.int)


    for user_index in range(URM_all.shape[0]):

        if user_index % 10000 == 0:
            print("split_data_on_sequence: user {} of {}".format(user_index, URM_all.shape[0]))

        start_pos = URM_all.indptr[user_index]
        end_pos = URM_all.indptr[user_index+1]

        user_profile = URM_all.indices[start_pos:end_pos]
        user_data = URM_all.data[start_pos:end_pos] 
#         implicit이므로 딱히 필요 없을지도..
        user_sequence = URM_timestamp.data[start_pos:end_pos]


        if len(user_profile) >= 3:

            # Test contain the first one, validation the second
            min_pos = np.argmax(user_sequence)

            venue_index = user_profile[min_pos]
            venue_data = user_data[min_pos] ##1

            URM_test_builder.add_data_lists([user_index], [venue_index], [venue_data])

            user_profile = np.delete(user_profile, min_pos)
            user_data = np.delete(user_data, min_pos)
            user_sequence = np.delete(user_sequence, min_pos)


            min_pos = np.argmax(user_sequence)

            venue_index = user_profile[min_pos]
            venue_data = user_data[min_pos]

            URM_validation_builder.add_data_lists([user_index], [venue_index], [venue_data])

            user_profile = np.delete(user_profile, min_pos)
            user_data = np.delete(user_data, min_pos)
#             user_sequence = np.delete(user_sequence, min_pos)


            URM_train_builder.add_data_lists([user_index]*len(user_profile), user_profile, user_data)


    URM_train = URM_train_builder.get_SparseMatrix()
    URM_validation = URM_validation_builder.get_SparseMatrix()
    URM_test = URM_test_builder.get_SparseMatrix()



    return URM_train, URM_validation, URM_test

In [12]:
URM_train, URM_validation, URM_test = split_data_on_timestamp(df_user_movie_rating_mtx,df_user_movie_timestamp_mtx)

(6040, 3706)
split_data_on_sequence: user 0 of 6040


In [13]:
from Evaluator.Evaluator import EvaluatorHoldout
from TopPopular import TopPopular
from check_matrix import check_matrix

In [14]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[5, 10, 20], exclude_seen=True)

In [15]:
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[5, 10, 20], exclude_seen=True)

In [64]:
recommender = TopPopular(URM_train)
recommender.fit()
results_dict = evaluator_validation.evaluateRecommender(recommender)
print(results_dict)

TopPopularRecommender: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.45 sec. Users per second: 4180
CUTOFF: 5 - AUROC: 0.0136175, NDCG: 0.0155677, HR: 0.0241722, 
CUTOFF: 10 - AUROC: 0.0239146, NDCG: 0.0226241, HR: 0.0465232, 
CUTOFF: 20 - AUROC: 0.0430986, NDCG: 0.0300127, HR: 0.0759934, 



--------------------------------------

In [21]:
from item_CF import *

In [47]:
recommender = item_CF(URM_train)
recommender.fit(feature_weighting = "BM25", similarity='asymmetric', topK=20)

item_CF: URM Detected 3 (0.08 %) cold items.


In [48]:
results_dict = evaluator_validation.evaluateRecommender(recommender)
print(results_dict)

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.74 sec. Users per second: 3467
CUTOFF: 5 - AUROC: 0.0295530, NDCG: 0.0327380, HR: 0.0524834, 
CUTOFF: 10 - AUROC: 0.0514165, NDCG: 0.0449467, HR: 0.0903974, 
CUTOFF: 20 - AUROC: 0.0855699, NDCG: 0.0586629, HR: 0.1450331, 



In [46]:
recommender = item_CF(URM_train)
top = [10, 20, 30, 50, 100]

for i in top:
    recommender.fit(feature_weighting = "TF_IDF", similarity="cosine", topK=i)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topk : ", i)
    print(results_dict)


item_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.98 sec. Users per second: 3057
topk :  10
CUTOFF: 5 - AUROC: 0.0310844, NDCG: 0.0346514, HR: 0.0544702, 
CUTOFF: 10 - AUROC: 0.0526122, NDCG: 0.0461907, HR: 0.0903974, 
CUTOFF: 20 - AUROC: 0.0871819, NDCG: 0.0605030, HR: 0.1471854, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.93 sec. Users per second: 3136
topk :  20
CUTOFF: 5 - AUROC: 0.0320364, NDCG: 0.0350815, HR: 0.0543046, 
CUTOFF: 10 - AUROC: 0.0532745, NDCG: 0.0477236, HR: 0.0938742, 
CUTOFF: 20 - AUROC: 0.0906675, NDCG: 0.0633306, HR: 0.1557947, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.75 sec. Users per second: 3451
topk :  30
CUTOFF: 5 - AUROC: 0.0289735, NDCG: 0.0335421, HR: 0.0541391, 
CUTOFF: 10 - AUROC: 0.0513429, NDCG: 0.0452540, HR: 0.0907285, 
CUTOFF: 20 - AUROC: 0.0871384, NDCG: 0.0607156, HR: 0.1524834, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.89 sec. Users per second: 3198
topk :  50
CUTOFF: 5 -

In [43]:
recommender = item_CF(URM_train)
similarity=["cosine","asymmetric","jaccard","tanimoto","dice","tversky" ]

for i in similarity:
    recommender.fit(feature_weighting = "TF_IDF", similarity=i, topK=20)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("similarity : ", i)
    print(results_dict)

item_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.69 sec. Users per second: 3582
similarity :  cosine
CUTOFF: 5 - AUROC: 0.0316225, NDCG: 0.0341942, HR: 0.0533113, 
CUTOFF: 10 - AUROC: 0.0525570, NDCG: 0.0464954, HR: 0.0917219, 
CUTOFF: 20 - AUROC: 0.0883060, NDCG: 0.0614914, HR: 0.1513245, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.65 sec. Users per second: 3659
similarity :  asymmetric
CUTOFF: 5 - AUROC: 0.0320364, NDCG: 0.0350815, HR: 0.0543046, 
CUTOFF: 10 - AUROC: 0.0532745, NDCG: 0.0477236, HR: 0.0938742, 
CUTOFF: 20 - AUROC: 0.0906675, NDCG: 0.0633306, HR: 0.1557947, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.58 sec. Users per second: 3822
similarity :  jaccard
CUTOFF: 5 - AUROC: 0.0278146, NDCG: 0.0317546, HR: 0.0519868, 
CUTOFF: 10 - AUROC: 0.0494849, NDCG: 0.0431663, HR: 0.0875828, 
CUTOFF: 20 - AUROC: 0.0829819, NDCG: 0.0571892, HR: 0.1437086, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.66 sec. Users per 

In [40]:
recommender = item_CF(URM_train)
recommender.fit(feature_weighting = "TF_IDF", similarity ='asymmetric', topK=20)
results_dict = evaluator_validation.evaluateRecommender(recommender)

print(results_dict)

item_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.67 sec. Users per second: 3625
CUTOFF: 5 - AUROC: 0.0316225, NDCG: 0.0341942, HR: 0.0533113, 
CUTOFF: 10 - AUROC: 0.0525570, NDCG: 0.0464954, HR: 0.0917219, 
CUTOFF: 20 - AUROC: 0.0883060, NDCG: 0.0614914, HR: 0.1513245, 



In [60]:
from P3alpha import *

In [65]:
recommender = P3alpha(URM_train)
recommender.fit(topK=50, alpha=0.5)

P3alpha: URM Detected 3 (0.08 %) cold items.


In [66]:
results_dict = evaluator_validation.evaluateRecommender(recommender)
print(results_dict)

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.63 sec. Users per second: 3696
CUTOFF: 5 - AUROC: 0.0261589, NDCG: 0.0286846, HR: 0.0448675, 
CUTOFF: 10 - AUROC: 0.0432487, NDCG: 0.0375463, HR: 0.0723510, 
CUTOFF: 20 - AUROC: 0.0687957, NDCG: 0.0477778, HR: 0.1130795, 



In [55]:
# recommender = item_CF(URM_train+URM_validation)
# recommender.fit(feature_weighting = "TF_IDF")
recommender = P3alpha(URM_train+URM_validation)
recommender.fit(topK=50, alpha=1)

P3alpha: URM Detected 3 (0.08 %) cold items.


In [56]:
results_dict = evaluator_test.evaluateRecommender(recommender)
results_dict

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.65 sec. Users per second: 3658


'CUTOFF: 5 - AUROC: 0.0260762, NDCG: 0.0287215, HR: 0.0443709, \nCUTOFF: 10 - AUROC: 0.0428808, NDCG: 0.0381405, HR: 0.0738411, \nCUTOFF: 20 - AUROC: 0.0686999, NDCG: 0.0475177, HR: 0.1110927, \n'

In [68]:
recommender = P3alpha(URM_train)
top=[10,20,30,50,100]
for j in top:
    for i in range(10):
        recommender.fit(topK=j, alpha=i*0.1)
        results_dict = evaluator_validation.evaluateRecommender(recommender)
        print("alpha : ", i*0.1)
        print(results_dict)

P3alpha: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.45 sec. Users per second: 4159
alpha :  0
CUTOFF: 5 - AUROC: 0.0222268, NDCG: 0.0243359, HR: 0.0370861, 
CUTOFF: 10 - AUROC: 0.0355592, NDCG: 0.0315738, HR: 0.0597682, 
CUTOFF: 20 - AUROC: 0.0589317, NDCG: 0.0426281, HR: 0.1039735, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.36 sec. Users per second: 4449
alpha :  1
CUTOFF: 5 - AUROC: 0.0221026, NDCG: 0.0244746, HR: 0.0385762, 
CUTOFF: 10 - AUROC: 0.0370125, NDCG: 0.0313952, HR: 0.0597682, 
CUTOFF: 20 - AUROC: 0.0547142, NDCG: 0.0372076, HR: 0.0829470, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.35 sec. Users per second: 4490
alpha :  2
CUTOFF: 5 - AUROC: 0.0068709, NDCG: 0.0074947, HR: 0.0120861, 
CUTOFF: 10 - AUROC: 0.0113870, NDCG: 0.0097298, HR: 0.0190397, 
CUTOFF: 20 - AUROC: 0.0187870, NDCG: 0.0128788, HR: 0.0314570, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.35 sec. Users per second: 4488
alpha :  3
CUTOFF: 5 -

KeyboardInterrupt: 

- HR기준 최적 알파: 0.7

In [69]:
recommender = P3alpha(URM_train)
top = [20, 30, 50, 100]
for i in top:
    recommender.fit(topK=i, alpha=0.3)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topk : ", i)
    print(results_dict)

P3alpha: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.41 sec. Users per second: 4294
topk :  20
CUTOFF: 5 - AUROC: 0.0254553, NDCG: 0.0274601, HR: 0.0403974, 
CUTOFF: 10 - AUROC: 0.0381531, NDCG: 0.0344179, HR: 0.0624172, 
CUTOFF: 20 - AUROC: 0.0612757, NDCG: 0.0454355, HR: 0.1066225, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.41 sec. Users per second: 4276
topk :  30
CUTOFF: 5 - AUROC: 0.0230546, NDCG: 0.0253171, HR: 0.0402318, 
CUTOFF: 10 - AUROC: 0.0383738, NDCG: 0.0337096, HR: 0.0665563, 
CUTOFF: 20 - AUROC: 0.0615371, NDCG: 0.0425469, HR: 0.1019868, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.48 sec. Users per second: 4077
topk :  50
CUTOFF: 5 - AUROC: 0.0234685, NDCG: 0.0260312, HR: 0.0407285, 
CUTOFF: 10 - AUROC: 0.0397535, NDCG: 0.0349779, HR: 0.0685430, 
CUTOFF: 20 - AUROC: 0.0637853, NDCG: 0.0442415, HR: 0.1056291, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.74 sec. Users per second: 3481
topk :  100
CUTOFF: 5 

In [70]:
recommender = P3alpha(URM_train)
top = [20, 30, 50, 100]
for i in top:
    recommender.fit(topK=i, alpha=0.8)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topk : ", i)
    print(results_dict)

P3alpha: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.73 sec. Users per second: 3500
topk :  20
CUTOFF: 5 - AUROC: 0.0280215, NDCG: 0.0309677, HR: 0.0476821, 
CUTOFF: 10 - AUROC: 0.0462656, NDCG: 0.0410071, HR: 0.0789735, 
CUTOFF: 20 - AUROC: 0.0751046, NDCG: 0.0529941, HR: 0.1268212, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.75 sec. Users per second: 3455
topk :  30
CUTOFF: 5 - AUROC: 0.0296772, NDCG: 0.0322875, HR: 0.0496689, 
CUTOFF: 10 - AUROC: 0.0469831, NDCG: 0.0419241, HR: 0.0801325, 
CUTOFF: 20 - AUROC: 0.0774137, NDCG: 0.0549045, HR: 0.1317881, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.51 sec. Users per second: 4005
topk :  50
CUTOFF: 5 - AUROC: 0.0287666, NDCG: 0.0318138, HR: 0.0495033, 
CUTOFF: 10 - AUROC: 0.0476269, NDCG: 0.0424319, HR: 0.0827815, 
CUTOFF: 20 - AUROC: 0.0784420, NDCG: 0.0552273, HR: 0.1339404, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.70 sec. Users per second: 3553
topk :  100
CUTOFF: 5 

In [50]:
from user_CF import *

In [59]:
recommender = user_CF(URM_train)
recommender.fit(feature_weighting = "BM25", similarity='cosine', topK=100)
results_dict = evaluator_validation.evaluateRecommender(recommender)
print(results_dict)

user_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.82 sec. Users per second: 3323
CUTOFF: 5 - AUROC: 0.0187500, NDCG: 0.0215433, HR: 0.0337748, 
CUTOFF: 10 - AUROC: 0.0332046, NDCG: 0.0297953, HR: 0.0594371, 
CUTOFF: 20 - AUROC: 0.0571628, NDCG: 0.0405400, HR: 0.1024834, 



In [56]:
recommender = user_CF(URM_train)
top = [10, 20, 30, 50, 100]

for i in top:
    recommender.fit(feature_weighting = "TF_IDF", similarity='cosine', topK=i)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topK : ", i)
    print(results_dict)

user_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.62 sec. Users per second: 3725
topK :  10
CUTOFF: 5 - AUROC: 0.0185844, NDCG: 0.0211376, HR: 0.0349338, 
CUTOFF: 10 - AUROC: 0.0332046, NDCG: 0.0292961, HR: 0.0605960, 
CUTOFF: 20 - AUROC: 0.0579906, NDCG: 0.0396547, HR: 0.1018212, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.49 sec. Users per second: 4056
topK :  20
CUTOFF: 5 - AUROC: 0.0223924, NDCG: 0.0252701, HR: 0.0399007, 
CUTOFF: 10 - AUROC: 0.0389993, NDCG: 0.0352863, HR: 0.0713576, 
CUTOFF: 20 - AUROC: 0.0687870, NDCG: 0.0478284, HR: 0.1211921, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.49 sec. Users per second: 4045
topK :  30
CUTOFF: 5 - AUROC: 0.0238411, NDCG: 0.0277777, HR: 0.0432119, 
CUTOFF: 10 - AUROC: 0.0422921, NDCG: 0.0391884, HR: 0.0791391, 
CUTOFF: 20 - AUROC: 0.0746514, NDCG: 0.0524218, HR: 0.1319536, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.61 sec. Users per second: 3757
topK :  50
CUTOFF: 5 -

In [57]:
recommender = user_CF(URM_train)
similarity=['cosine', 'asymmetric','jaccard','tanimoto','dice','tversky' ]

for i in similarity:
    recommender.fit(feature_weighting = "TF_IDF", similarity= i , topK=100)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("similarity : ", i)
    print(results_dict)

user_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.80 sec. Users per second: 3353
similarity :  cosine
CUTOFF: 5 - AUROC: 0.0275662, NDCG: 0.0308771, HR: 0.0473510, 
CUTOFF: 10 - AUROC: 0.0472406, NDCG: 0.0430483, HR: 0.0854305, 
CUTOFF: 20 - AUROC: 0.0823980, NDCG: 0.0588494, HR: 0.1486755, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.80 sec. Users per second: 3351
similarity :  asymmetric
CUTOFF: 5 - AUROC: 0.0278974, NDCG: 0.0314342, HR: 0.0478477, 
CUTOFF: 10 - AUROC: 0.0476821, NDCG: 0.0433815, HR: 0.0850993, 
CUTOFF: 20 - AUROC: 0.0825810, NDCG: 0.0590503, HR: 0.1476821, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.74 sec. Users per second: 3463
similarity :  jaccard
CUTOFF: 5 - AUROC: 0.0237583, NDCG: 0.0267887, HR: 0.0415563, 
CUTOFF: 10 - AUROC: 0.0412252, NDCG: 0.0372874, HR: 0.0743377, 
CUTOFF: 20 - AUROC: 0.0731265, NDCG: 0.0515549, HR: 0.1309603, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.73 sec. Users per 

In [69]:
from RP3beta import *

In [71]:
recommender = RP3beta(URM_train)
top=[10,20,30,50,100]
for j in top:
    for i in range(10):
        recommender.fit(topK=j, alpha=0.9, beta=i*0.1)
        results_dict = evaluator_validation.evaluateRecommender(recommender)
        print("topK :",j," beta : ", i*0.1)
        print(results_dict)

RP3beta: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.52 sec. Users per second: 3974
topK : 10  beta :  0.0
CUTOFF: 5 - AUROC: 0.0283526, NDCG: 0.0301350, HR: 0.0453642, 
CUTOFF: 10 - AUROC: 0.0442421, NDCG: 0.0392439, HR: 0.0738411, 
CUTOFF: 20 - AUROC: 0.0727344, NDCG: 0.0516195, HR: 0.1228477, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.48 sec. Users per second: 4074
topK : 10  beta :  0.1
CUTOFF: 5 - AUROC: 0.0281043, NDCG: 0.0309038, HR: 0.0470199, 
CUTOFF: 10 - AUROC: 0.0445364, NDCG: 0.0398803, HR: 0.0753311, 
CUTOFF: 20 - AUROC: 0.0754183, NDCG: 0.0537101, HR: 0.1299669, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.43 sec. Users per second: 4227
topK : 10  beta :  0.2
CUTOFF: 5 - AUROC: 0.0295530, NDCG: 0.0325182, HR: 0.0495033, 
CUTOFF: 10 - AUROC: 0.0476637, NDCG: 0.0429372, HR: 0.0822848, 
CUTOFF: 20 - AUROC: 0.0798188, NDCG: 0.0562356, HR: 0.1349338, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.46 sec. Users per

topK : 30  beta :  0.7000000000000001
CUTOFF: 5 - AUROC: 0.0174669, NDCG: 0.0207172, HR: 0.0355960, 
CUTOFF: 10 - AUROC: 0.0342899, NDCG: 0.0305672, HR: 0.0665563, 
CUTOFF: 20 - AUROC: 0.0647002, NDCG: 0.0442587, HR: 0.1211921, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.73 sec. Users per second: 3492
topK : 30  beta :  0.8
CUTOFF: 5 - AUROC: 0.0082781, NDCG: 0.0105038, HR: 0.0193709, 
CUTOFF: 10 - AUROC: 0.0185430, NDCG: 0.0167473, HR: 0.0390728, 
CUTOFF: 20 - AUROC: 0.0415737, NDCG: 0.0288973, HR: 0.0874172, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.77 sec. Users per second: 3415
topK : 30  beta :  0.9
CUTOFF: 5 - AUROC: 0.0017384, NDCG: 0.0020724, HR: 0.0043046, 
CUTOFF: 10 - AUROC: 0.0055556, NDCG: 0.0053711, HR: 0.0145695, 
CUTOFF: 20 - AUROC: 0.0163036, NDCG: 0.0122583, HR: 0.0423841, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.47 sec. Users per second: 4122
topK : 50  beta :  0.0
CUTOFF: 5 - AUROC: 0.0298427, NDCG: 0.0325433, HR: 0.0498344, 
CUTOFF: 10 

In [72]:
recommender = RP3beta(URM_train)
for i in range(0, 10):
    recommender.fit(topK=50, alpha=0.8, beta=i*0.1)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print('beta : ', i*0.1)
    print(results_dict)

RP3beta: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.76 sec. Users per second: 3438
beta :  0.0
CUTOFF: 5 - AUROC: 0.0287666, NDCG: 0.0318138, HR: 0.0495033, 
CUTOFF: 10 - AUROC: 0.0476269, NDCG: 0.0424319, HR: 0.0827815, 
CUTOFF: 20 - AUROC: 0.0784420, NDCG: 0.0552273, HR: 0.1339404, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.49 sec. Users per second: 4066
beta :  0.1
CUTOFF: 5 - AUROC: 0.0298013, NDCG: 0.0330262, HR: 0.0516556, 
CUTOFF: 10 - AUROC: 0.0491538, NDCG: 0.0435710, HR: 0.0847682, 
CUTOFF: 20 - AUROC: 0.0808644, NDCG: 0.0569105, HR: 0.1380795, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.50 sec. Users per second: 4014
beta :  0.2
CUTOFF: 5 - AUROC: 0.0310017, NDCG: 0.0339598, HR: 0.0523179, 
CUTOFF: 10 - AUROC: 0.0507358, NDCG: 0.0450366, HR: 0.0869205, 
CUTOFF: 20 - AUROC: 0.0834611, NDCG: 0.0590565, HR: 0.1428808, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.52 sec. Users per second: 3982
beta :  0.300000000

In [83]:
recommender = TopPopular(URM_train)
recommender.fit()
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

TopPopularRecommender: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.51 sec. Users per second: 3997
CUTOFF: 5 - AUROC: 0.0150662, NDCG: 0.0169048, HR: 0.0274834, 
CUTOFF: 10 - AUROC: 0.0257726, NDCG: 0.0223544, HR: 0.0445364, 
CUTOFF: 20 - AUROC: 0.0430289, NDCG: 0.0300541, HR: 0.0753311, 



In [84]:
recommender = item_CF(URM_train)
recommender.fit(feature_weighting = "TF_IDF", similarity='asymmetric', topK=20)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

item_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.68 sec. Users per second: 3590
CUTOFF: 5 - AUROC: 0.0232616, NDCG: 0.0262371, HR: 0.0418874, 
CUTOFF: 10 - AUROC: 0.0403422, NDCG: 0.0361440, HR: 0.0730132, 
CUTOFF: 20 - AUROC: 0.0696846, NDCG: 0.0492550, HR: 0.1256623, 



In [85]:
recommender = user_CF(URM_train)
recommender.fit(feature_weighting = "TF_IDF", similarity='cosine', topK=100)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

user_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.78 sec. Users per second: 3386
CUTOFF: 5 - AUROC: 0.0201159, NDCG: 0.0232164, HR: 0.0374172, 
CUTOFF: 10 - AUROC: 0.0372333, NDCG: 0.0331020, HR: 0.0680464, 
CUTOFF: 20 - AUROC: 0.0668961, NDCG: 0.0471832, HR: 0.1243377, 



In [86]:
recommender = P3alpha(URM_train)
recommender.fit(topK=50, alpha=0.9)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

P3alpha: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.48 sec. Users per second: 4092
CUTOFF: 5 - AUROC: 0.0272351, NDCG: 0.0315711, HR: 0.0496689, 
CUTOFF: 10 - AUROC: 0.0464312, NDCG: 0.0407688, HR: 0.0783113, 
CUTOFF: 20 - AUROC: 0.0753486, NDCG: 0.0530141, HR: 0.1271523, 



In [87]:
recommender = RP3beta(URM_train)
recommender.fit(topK=50, alpha=0.9, beta=0.4)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

RP3beta: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.47 sec. Users per second: 4112
CUTOFF: 5 - AUROC: 0.0261589, NDCG: 0.0294384, HR: 0.0470199, 
CUTOFF: 10 - AUROC: 0.0446652, NDCG: 0.0389876, HR: 0.0768212, 
CUTOFF: 20 - AUROC: 0.0754357, NDCG: 0.0533569, HR: 0.1342715, 



In [92]:
recommender = TopPopular(URM_train+URM_validation)
recommender.fit()
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

TopPopularRecommender: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.43 sec. Users per second: 4212
CUTOFF: 5 - AUROC: 0.0154387, NDCG: 0.0174493, HR: 0.0283113, 
CUTOFF: 10 - AUROC: 0.0261221, NDCG: 0.0226665, HR: 0.0447020, 
CUTOFF: 20 - AUROC: 0.0434646, NDCG: 0.0304991, HR: 0.0759934, 



In [88]:
recommender = item_CF(URM_train+URM_validation)
recommender.fit(feature_weighting = "TF_IDF", similarity='asymmetric', topK=100)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

item_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.35 sec. Users per second: 2572
CUTOFF: 5 - AUROC: 0.0271109, NDCG: 0.0312878, HR: 0.0506623, 
CUTOFF: 10 - AUROC: 0.0487307, NDCG: 0.0424532, HR: 0.0852649, 
CUTOFF: 20 - AUROC: 0.0822151, NDCG: 0.0560787, HR: 0.1392384, 



In [89]:
recommender = user_CF(URM_train+URM_validation)
recommender.fit(feature_weighting = "TF_IDF", similarity='cosine', topK=100)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

user_CF: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.00 sec. Users per second: 3023
CUTOFF: 5 - AUROC: 0.0270281, NDCG: 0.0301651, HR: 0.0466887, 
CUTOFF: 10 - AUROC: 0.0452907, NDCG: 0.0404883, HR: 0.0789735, 
CUTOFF: 20 - AUROC: 0.0776142, NDCG: 0.0551180, HR: 0.1372517, 



In [90]:
recommender = P3alpha(URM_train+URM_validation)
recommender.fit(topK=50, alpha=1)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

P3alpha: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.53 sec. Users per second: 3944
CUTOFF: 5 - AUROC: 0.0293046, NDCG: 0.0320044, HR: 0.0504967, 
CUTOFF: 10 - AUROC: 0.0489882, NDCG: 0.0429436, HR: 0.0846026, 
CUTOFF: 20 - AUROC: 0.0800192, NDCG: 0.0557398, HR: 0.1357616, 



In [91]:
recommender = RP3beta(URM_train+URM_validation)
recommender.fit(topK=50, alpha=0.9, beta=0.4)
results_dict = evaluator_test.evaluateRecommender(recommender)
print(results_dict)

RP3beta: URM Detected 3 (0.08 %) cold items.
EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.50 sec. Users per second: 4014
CUTOFF: 5 - AUROC: 0.0298841, NDCG: 0.0337502, HR: 0.0531457, 
CUTOFF: 10 - AUROC: 0.0502575, NDCG: 0.0443298, HR: 0.0862583, 
CUTOFF: 20 - AUROC: 0.0832694, NDCG: 0.0590578, HR: 0.1451987, 

